In [ ]:
# LWVB: Larry Williams Volatility Breakout

import cryptocompare
import time
import datetime
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import pprint

import itertools
import tqdm

In [ ]:
targetDataFile = 'Upbit_BTC_KRW_hour_gmt.csv'
rate_fee = 0.003
rate_slippage = 0.000
rate_cost = rate_fee + rate_slippage

In [ ]:
dt_parser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z")

hist_hour = pd.read_csv(
    targetDataFile, 
    index_col = 0, 
    parse_dates = True)

# UTC + 9로 시간 변환
#hist_hour.index = hist_hour.index + datetime.timedelta(hours=9)
hist_hour.index = hist_hour.index.tz_convert('Asia/Seoul')

In [ ]:
def do_LWVB_backtest(hist_hour, name, begin_hour, func_MT, param_MT, func_TR, param_TR):
    hist_day = hist_hour.resample('1D', offset=datetime.timedelta(hours=begin_hour)).agg({
        'open': 'first', #lambda df: None if df.empty else df[0],
        'close': 'last', #lambda df: None if df.empty else df,
        'high': 'max',
        'low': 'min',
        'volume': 'sum',
        'value': 'sum',
    })

    func_MT(hist_day, param_MT)
    func_TR(hist_day, param_TR)

    hist_day['assets_after_sell'] = hist_day['rate_of_return'].cumprod()
    hist_day['assets_max'] = hist_day['assets_after_sell'].cummax()
    hist_day['dd'] = 1 - hist_day['assets_after_sell'] / hist_day['assets_max']
    #hist_day['underwater_times'] = 0
    #for idx, val in hist_day['dd'].iteritems():
    #    if val > 0 and idx > 0:
    #        hist_day['underwater_times'].iloc[idx] = hist_day['underwater_times'].iloc[idx-1] + 1
    hist_day['rate_of_return_percent'] = (hist_day['rate_of_return'] - 1) * 100
    hist_day['rate_of_return_percent_profit'] = np.where(hist_day['rate_of_return_percent'] > 0, hist_day['rate_of_return_percent'], np.nan)
    hist_day['rate_of_return_percent_loss'] = np.where(hist_day['rate_of_return_percent'] < 0, -hist_day['rate_of_return_percent'], np.nan)
    
    statistic = {
        'name': name,
        'begin_hour': begin_hour,
        'func_MT': func_MT.__name__,
        'param_MT': param_MT,
        'func_TR': func_TR.__name__,
        'param_TR': param_TR,
        ########################################################################
        'cagr(%)': hist_day['assets_after_sell'][-1] ** (1/(len(hist_day.index)/365)) * 100 - 100,
        'finalAsset': hist_day['assets_after_sell'][-1],
        'stdev(%)': np.std(hist_day['rate_of_return']) * 100, # -100 하면 안된다. ax+b의 sigma는 ax이다
        'mdd(%)' : np.max(hist_day['dd']) * 100,
        'ratio_win(%)' : np.sum(np.where(hist_day['rate_of_return'] > 1, 1, 0)) / np.sum(hist_day['trade']) * 100,
        'mean_profit(%)' : np.nanmean(hist_day['rate_of_return_percent_profit']),
        'mean_loss(%)' : np.nanmean(hist_day['rate_of_return_percent_loss']),
        'max_profit(%)' : np.nanmax(hist_day['rate_of_return_percent_profit']),
        'max_loss(%)' : np.nanmax(hist_day['rate_of_return_percent_loss']),
        #nderwater_times': np.max(hist_day['underwater_times']),
        'Num Of Trade' : np.sum(hist_day['trade']),
    }
    return statistic, hist_day['assets_after_sell'], hist_day['dd']



def MT_NoMT(hist_day, param_MT):
    hist_day['is_bull'] = True

def MT_maopen_and(hist_day, param_MT):
    if 'is_bull' not in hist_day.columns:
        hist_day['is_bull'] = True
    for open_dcount in param_MT['open_dcount_set']:
        colStr = 'maopen_{}'.format(open_dcount)
        hist_day[colStr] = hist_day['open'].shift(1).rolling(open_dcount).mean()
        hist_day['is_bull'] = hist_day['is_bull'] & (hist_day['close'].shift(1) > hist_day[colStr])

def MT_mavolume_and(hist_day, param_MT):
    if 'is_bull' not in hist_day.columns:
        hist_day['is_bull'] = True
    for vol_dcount in param_MT['vol_dcount_set']:
        colStr = 'mavolume_{}'.format(vol_dcount)
        hist_day[colStr] = hist_day['volume'].shift(1).rolling(vol_dcount).mean()
        hist_day['is_bull'] = hist_day['is_bull'] & (hist_day['volume'].shift(1) > hist_day[colStr])

def MT_maopen_mavolume_and(hist_day, param_MT):
    MT_maopen_and(hist_day, param_MT)
    MT_mavolume_and(hist_day, param_MT)




def TR_NaiveHold(hist_day, param_TR):
    hist_day['rate_of_return'] = np.where(hist_day['is_bull'], hist_day['close'] / hist_day['open'], 1)
    hist_day['trade'] = np.where(hist_day['is_bull'], 1, 0)

def TR_LWVB(hist_day, param_TR):
    hist_day['range_prev'] = hist_day['high'].shift(1) - hist_day['low'].shift(1)
    hist_day['breakout_price'] = hist_day['open'] + hist_day['range_prev'] * param_TR['break_rate']
    hist_day['breakout'] = hist_day['high'] > hist_day['breakout_price']
    hist_day['rate_of_return'] = np.where( \
        hist_day['is_bull'] & hist_day['breakout'], \
        hist_day['close'] / hist_day['breakout_price'] - rate_cost, 1)
    hist_day['trade'] = np.where(hist_day['is_bull'] & hist_day['breakout'], 1, 0)

def TR_LWVB_V(hist_day, param_TR):
    hist_day['range_prev'] = hist_day['high'].shift(1) - hist_day['low'].shift(1)
    hist_day['breakout_price'] = hist_day['open'] + hist_day['range_prev'] * param_TR['break_rate']
    hist_day['breakout'] = hist_day['high'] > hist_day['breakout_price']
    hist_day['volatility'] = hist_day['range_prev'] / hist_day['open'].shift(1)
    hist_day['invest_ratio'] = np.where( \
        hist_day['volatility'] <= param_TR['target_volatility'], \
        1, param_TR['target_volatility'] / hist_day['volatility'])
    hist_day['rate_of_return'] = \
        (1 - hist_day['invest_ratio']) + \
        hist_day['invest_ratio'] * np.where( \
            hist_day['is_bull'] & hist_day['breakout'], \
            hist_day['close'] / hist_day['breakout_price'] - rate_cost, 1)
    hist_day['trade'] = np.where(hist_day['is_bull'] & hist_day['breakout'], 1, 0)

def TR_LWVB_VN(hist_day, param_TR):
    hist_day['range_prev'] = hist_day['high'].shift(1) - hist_day['low'].shift(1)
    hist_day['noise_prev'] = 1 - np.abs(hist_day['close'].shift(1) - hist_day['open'].shift(1)) / (hist_day['high'].shift(1) - hist_day['low'].shift(1))
    hist_day['breakout_price'] = hist_day['open'] + hist_day['range_prev'] * hist_day['noise_prev']
    hist_day['breakout'] = hist_day['high'] > hist_day['breakout_price']
    hist_day['volatility'] = hist_day['range_prev'] / hist_day['open'].shift(1)
    hist_day['invest_ratio'] = np.where( \
        hist_day['volatility'] <= param_TR['target_volatility'], \
        1, param_TR['target_volatility'] / hist_day['volatility'])
    hist_day['rate_of_return'] = \
        (1 - hist_day['invest_ratio']) + \
            hist_day['invest_ratio'] * np.where( \
                hist_day['is_bull'] & hist_day['breakout'], \
                hist_day['close'] / hist_day['breakout_price'] - rate_cost, 1)
    hist_day['trade'] = np.where(hist_day['is_bull'] & hist_day['breakout'], 1, 0)



results_index = ['name', 'begin_hour', 'func_MT', 'param_MT','func_TR', 'param_TR', ]
results = pd.DataFrame(columns=results_index)

results.set_index(results_index, inplace=True)


MT_Set = [
    (MT_NoMT, {}),
    (MT_maopen_and, {'open_dcount_set': [3]}),
    (MT_maopen_and, {'open_dcount_set': [5]}),
    (MT_maopen_and, {'open_dcount_set': [10]}),
    (MT_maopen_and, {'open_dcount_set': [3, 5]}),
    (MT_maopen_and, {'open_dcount_set': [5, 10]}),
    (MT_maopen_and, {'open_dcount_set': [3, 5, 10]}),
    (MT_mavolume_and, {'vol_dcount_set': [3]}),
    (MT_mavolume_and, {'vol_dcount_set': [5]}),
    (MT_mavolume_and, {'vol_dcount_set': [10]}),
    (MT_mavolume_and, {'vol_dcount_set': [3, 5]}),
    (MT_mavolume_and, {'vol_dcount_set': [5, 10]}),
    (MT_mavolume_and, {'vol_dcount_set': [3, 5, 10]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [3], 'vol_dcount_set': [2]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [5], 'vol_dcount_set': [2]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [10], 'vol_dcount_set': [2]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [3], 'vol_dcount_set': [3]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [5], 'vol_dcount_set': [3]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [10], 'vol_dcount_set': [3]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [3], 'vol_dcount_set': [5]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [5], 'vol_dcount_set': [5]}),
    (MT_maopen_mavolume_and, {'open_dcount_set': [10], 'vol_dcount_set': [5]}),
]
TR_Set = [
    (TR_NaiveHold, {}),
    (TR_LWVB, {'break_rate': 0.1}),
    (TR_LWVB, {'break_rate': 0.2}),
    (TR_LWVB, {'break_rate': 0.3}),
    (TR_LWVB, {'break_rate': 0.4}),
    (TR_LWVB, {'break_rate': 0.5}),
    (TR_LWVB, {'break_rate': 0.6}),
    (TR_LWVB, {'break_rate': 0.7}),
    (TR_LWVB, {'break_rate': 0.8}),
    (TR_LWVB, {'break_rate': 0.9}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.005}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.01}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.02}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.03}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.05}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.07}),
    (TR_LWVB_V, {'break_rate': 0.3, 'target_volatility': 0.10}),

    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.005}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.01}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.02}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.03}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.05}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.07}),
    (TR_LWVB_V, {'break_rate': 0.5, 'target_volatility': 0.10}),

    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.005}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.01}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.02}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.03}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.05}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.07}),
    (TR_LWVB_V, {'break_rate': 0.7, 'target_volatility': 0.10}),

    (TR_LWVB_VN, {'target_volatility': 0.005}),
    (TR_LWVB_VN, {'target_volatility': 0.01}),
    (TR_LWVB_VN, {'target_volatility': 0.02}),
    (TR_LWVB_VN, {'target_volatility': 0.03}),
    (TR_LWVB_VN, {'target_volatility': 0.05}),
    (TR_LWVB_VN, {'target_volatility': 0.07}),
    (TR_LWVB_VN, {'target_volatility': 0.10}),
]

strategies = itertools.product(range(0, 24), MT_Set, TR_Set) 
for begin_hour, MT_Pair, TR_Pair in tqdm.tqdm(strategies, total=24*len(MT_Set)*len(TR_Set)): 
    ret, _, _ = do_LWVB_backtest(hist_hour, '', begin_hour, MT_Pair[0], MT_Pair[1], TR_Pair[0], TR_Pair[1])
    results = results.append(ret, ignore_index=True)

graph = pd.DataFrame()
px.line(
    graph.filter(regex='assets.*'), 
    log_y=True).update_layout(showlegend=True).show()
px.line(
    graph.filter(regex='dd.*')).update_layout(showlegend=True).show()


results.to_csv('results.csv')

In [ ]:
for a, b in itertools.product([1, 2, 3], itertools.product([6, 7], [-1, -2])):
    print (a, b)

In [ ]:
begin_hour, static_break_rate = 0, 0.5

hist_day = hist_hour.resample('1D', offset=datetime.timedelta(hours=begin_hour)).agg({
    'open': 'first', #lambda df: None if df.empty else df[0],
    'close': 'last', #lambda df: None if df.empty else df,
    'high': 'max',
    'low': 'min',
    'volume': 'sum',
    'value': 'sum',
})

# common
hist_day['ma3_open'] = hist_day['open'].shift(1).rolling(3).mean()
hist_day['ma5_open'] = hist_day['open'].shift(1).rolling(5).mean()
hist_day['ma10_open'] = hist_day['open'].shift(1).rolling(10).mean()
hist_day['ma3_volume'] = hist_day['volume'].shift(1).rolling(3).mean()
hist_day['ma5_volume'] = hist_day['volume'].shift(1).rolling(5).mean()
hist_day['ma10_volume'] = hist_day['volume'].shift(1).rolling(10).mean()
hist_day['range_prev'] = hist_day['high'].shift(1) - hist_day['low'].shift(1)
hist_day['breakout_price_static'] = hist_day['open'] + hist_day['range_prev'] * static_break_rate
hist_day['breakout'] = hist_day['high'] > hist_day['breakout_price_static']

days = len(hist_day.index)


# Naive Hold
hist_day['Naive.rate_of_return'] =  hist_day['close'] / hist_day['open']
hist_day['Naive.assets_after_sell'] = hist_day['Naive.rate_of_return'].cumprod()
hist_day['Naive.assets_max'] = hist_day['Naive.assets_after_sell'].cummax()
hist_day['Naive.dd'] = 1 - hist_day['Naive.assets_after_sell'] / hist_day['Naive.assets_max']
hist_day['Naive.rate_of_return_percent'] = (hist_day['Naive.rate_of_return'] - 1) * 100
hist_day['Naive.rate_of_return_percent_profit'] = np.where(hist_day['Naive.rate_of_return_percent'] > 0, hist_day['Naive.rate_of_return_percent'], np.nan)
hist_day['Naive.rate_of_return_percent_loss'] = np.where(hist_day['Naive.rate_of_return_percent'] < 0, -hist_day['Naive.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "Naive.final_asset" : hist_day['Naive.assets_after_sell'][-1],
    "Naive.cagr(%)" : hist_day['Naive.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "Naive.mdd(%)" : np.max(hist_day['Naive.dd']) * 100,
    "Naive.ratio_win(%)" : np.sum(np.where(hist_day['Naive.rate_of_return'] > 1, 1, 0)) / len(hist_day['Naive.rate_of_return']) * 100,
    "Naive.mean_profit(%)" : np.nanmean(hist_day['Naive.rate_of_return_percent_profit']),
    "Naive.mean_loss(%)" : np.nanmean(hist_day['Naive.rate_of_return_percent_loss']),
})


# LWVB (classic)
hist_day['LWVB.rate_of_return'] = np.where(hist_day['breakout'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) # breakout 안했으면 자산 유지
hist_day['LWVB.assets_after_sell'] = hist_day['LWVB.rate_of_return'].cumprod()
hist_day['LWVB.assets_max'] = hist_day['LWVB.assets_after_sell'].cummax()
hist_day['LWVB.dd'] = 1 - hist_day['LWVB.assets_after_sell'] / hist_day['LWVB.assets_max']
hist_day['LWVB.rate_of_return_percent'] = (hist_day['LWVB.rate_of_return'] - 1) * 100
hist_day['LWVB.rate_of_return_percent_profit'] = np.where(hist_day['LWVB.rate_of_return_percent'] > 0, hist_day['LWVB.rate_of_return_percent'], np.nan)
hist_day['LWVB.rate_of_return_percent_loss'] = np.where(hist_day['LWVB.rate_of_return_percent'] < 0, -hist_day['LWVB.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB.final_asset" : hist_day['LWVB.assets_after_sell'][-1],
    "LWVB.cagr(%)" : hist_day['LWVB.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB.mdd(%)" : np.max(hist_day['LWVB.dd']) * 100,
    "LWVB.ratio_win(%)" : np.sum(np.where(hist_day['LWVB.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['breakout'], 1, 0)) * 100,
    "LWVB.mean_profit(%)" : np.nanmean(hist_day['LWVB.rate_of_return_percent_profit']),
    "LWVB.mean_loss(%)" : np.nanmean(hist_day['LWVB.rate_of_return_percent_loss']),
})

# LWVB with MT(ma5_open)
hist_day['LWVB_MT1.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open'])
hist_day['LWVB_MT1.rate_of_return'] = np.where(hist_day['LWVB_MT1.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) 
hist_day['LWVB_MT1.assets_after_sell'] = hist_day['LWVB_MT1.rate_of_return'].cumprod()
hist_day['LWVB_MT1.assets_max'] = hist_day['LWVB_MT1.assets_after_sell'].cummax()
hist_day['LWVB_MT1.dd'] = 1 - hist_day['LWVB_MT1.assets_after_sell'] / hist_day['LWVB_MT1.assets_max']
hist_day['LWVB_MT1.rate_of_return_percent'] = (hist_day['LWVB_MT1.rate_of_return'] - 1) * 100
hist_day['LWVB_MT1.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_MT1.rate_of_return_percent'] > 0, hist_day['LWVB_MT1.rate_of_return_percent'], np.nan)
hist_day['LWVB_MT1.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_MT1.rate_of_return_percent'] < 0, -hist_day['LWVB_MT1.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_MT1.final_asset" : hist_day['LWVB_MT1.assets_after_sell'][-1],
    "LWVB_MT1.cagr(%)" : hist_day['LWVB_MT1.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_MT1.mdd(%)" : np.max(hist_day['LWVB_MT1.dd']) * 100,
    "LWVB_MT1.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_MT1.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_MT1.is_bull'], 1, 0)) * 100,
    "LWVB_MT1.mean_profit(%)" : np.nanmean(hist_day['LWVB_MT1.rate_of_return_percent_profit']),
    "LWVB_MT1.mean_loss(%)" : np.nanmean(hist_day['LWVB_MT1.rate_of_return_percent_loss']),
})
    
# LWVB_MT2
hist_day['LWVB_MT2.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open']) & (hist_day['volume'] > hist_day['ma3_volume']) 
hist_day['LWVB_MT2.rate_of_return'] = np.where(hist_day['LWVB_MT2.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1) 
hist_day['LWVB_MT2.assets_after_sell'] = hist_day['LWVB_MT2.rate_of_return'].cumprod()
hist_day['LWVB_MT2.assets_max'] = hist_day['LWVB_MT2.assets_after_sell'].cummax()
hist_day['LWVB_MT2.dd'] = 1 - hist_day['LWVB_MT2.assets_after_sell'] / hist_day['LWVB_MT2.assets_max']
hist_day['LWVB_MT2.rate_of_return_percent'] = (hist_day['LWVB_MT2.rate_of_return'] - 1) * 100
hist_day['LWVB_MT2.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_MT2.rate_of_return_percent'] > 0, hist_day['LWVB_MT2.rate_of_return_percent'], np.nan)
hist_day['LWVB_MT2.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_MT2.rate_of_return_percent'] < 0, -hist_day['LWVB_MT2.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_MT2.final_asset" : hist_day['LWVB_MT2.assets_after_sell'][-1],
    "LWVB_MT2.cagr(%)" : hist_day['LWVB_MT2.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_MT2.mdd(%)" : np.max(hist_day['LWVB_MT2.dd']) * 100,
    "LWVB_MT2.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_MT2.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_MT2.is_bull'], 1, 0)) * 100,
    "LWVB_MT2.mean_profit(%)" : np.nanmean(hist_day['LWVB_MT2.rate_of_return_percent_profit']),
    "LWVB_MT2.mean_loss(%)" : np.nanmean(hist_day['LWVB_MT2.rate_of_return_percent_loss']),
})

# LWVB_V : LWVB with volatility control
hist_day['LWVB_V.is_bull'] = (hist_day['breakout']) & (hist_day['open'] > hist_day['ma5_open']) & (hist_day['volume'] > hist_day['ma3_volume'])
hist_day['LWVB_V.volatility'] = hist_day['range_prev'] / hist_day['open'].shift(1)
hist_day['LWVB_V.invest_ratio'] = np.where(hist_day['LWVB_V.volatility'] <= 0.01, 1, 0.01 / hist_day['LWVB_V.volatility'])
hist_day['LWVB_V.rate_of_return'] = \
    (1 - hist_day['LWVB_V.invest_ratio']) + \
        hist_day['LWVB_V.invest_ratio'] * \
            np.where(hist_day['LWVB_V.is_bull'], hist_day['close'] / hist_day['breakout_price_static'] - rate_cost, 1)
hist_day['LWVB_V.assets_after_sell'] = hist_day['LWVB_V.rate_of_return'].cumprod()
hist_day['LWVB_V.assets_max'] = hist_day['LWVB_V.assets_after_sell'].cummax()
hist_day['LWVB_V.dd'] = 1 - hist_day['LWVB_V.assets_after_sell'] / hist_day['LWVB_V.assets_max']
hist_day['LWVB_V.rate_of_return_percent'] = (hist_day['LWVB_V.rate_of_return'] - 1) * 100
hist_day['LWVB_V.rate_of_return_percent_profit'] = np.where(hist_day['LWVB_V.rate_of_return_percent'] > 0, hist_day['LWVB_V.rate_of_return_percent'], np.nan)
hist_day['LWVB_V.rate_of_return_percent_loss'] = np.where(hist_day['LWVB_V.rate_of_return_percent'] < 0, -hist_day['LWVB_V.rate_of_return_percent'], np.nan)
pprint.pprint( {
    "LWVB_V.final_asset" : hist_day['LWVB_V.assets_after_sell'][-1],
    "LWVB_V.cagr(%)" : hist_day['LWVB_V.assets_after_sell'][-1] ** (1/(days/365)) * 100 - 100,
    "LWVB_V.mdd(%)" : np.max(hist_day['LWVB_V.dd']) * 100,
    "LWVB_V.ratio_win(%)" : np.sum(np.where(hist_day['LWVB_V.rate_of_return'] > 1, 1, 0)) / np.sum(np.where(hist_day['LWVB_V.is_bull'], 1, 0)) * 100,
    "LWVB_V.mean_profit(%)" : np.nanmean(hist_day['LWVB_V.rate_of_return_percent_profit']),
    "LWVB_V.mean_loss(%)" : np.nanmean(hist_day['LWVB_V.rate_of_return_percent_loss']),
})

px.line(
    hist_day, 
    y=[
        'Naive.assets_after_sell',
        'LWVB.assets_after_sell',
        'LWVB_MT1.assets_after_sell',
        'LWVB_MT2.assets_after_sell',
        'LWVB_V.assets_after_sell'], 
    log_y=True).show()
px.line(
    hist_day, 
    y=[
        'Naive.dd',
        'LWVB.dd',
        'LWVB_MT1.dd',
        'LWVB_MT2.dd',
        'LWVB_V.dd']
    ).show()


hist_day.to_csv(targetDataFile + "_result_LWVB.csv")